##### To submit jobs for terrain correction to Alaska Satellite Facility's HyP3 API

In [19]:
import pandas as pd
import os
import hyp3_sdk as sdk
from hyp3_sdk import HyP3
import getpass

In [ ]:
uid = getpass.getpass("Enter your EathData user id?")
pswd = getpass.getpass("Enter your EathData password?")
hyp3 = HyP3(username=uid, password=pswd)

In [4]:
DATA_DIR = "./files_list"
an_df = pd.read_csv(os.path.join(DATA_DIR, "s1_an.csv"), names=["ID", "Path"])
au_df = pd.read_csv(os.path.join(DATA_DIR, "s1_au.csv"), names=["ID", "Path"])

AN_ID = an_df.ID.unique()[2]
AU_ID = au_df.ID.unique()[1]

an_df = an_df[an_df.ID == AN_ID].reset_index(drop=True)
au_df = au_df[au_df.ID == AU_ID].reset_index(drop=True)

an_df = an_df[an_df.Path.apply(lambda x: ("SLC" in x) and ("IW" in x))].reset_index(
    drop=True
)
au_df = au_df[au_df.Path.apply(lambda x: ("SLC" in x) and ("IW" in x))].reset_index(
    drop=True
)

grouper = an_df.Path.apply(lambda r: os.path.split(r)[1].split("_")[5][0:6])
AN_SCENE_LIST = [
    list(an_df.groupby(grouper))[i][1].Path.iloc[0]
    for i in range(0, len(grouper.unique()))
]

grouper = au_df.Path.apply(lambda r: os.path.split(r)[1].split("_")[5][0:6])
AU_SCENE_LIST = [
    list(au_df.groupby(grouper))[i][1].Path.iloc[0]
    for i in range(0, len(grouper.unique()))
]

SCENE_LIST = AN_SCENE_LIST + AU_SCENE_LIST

SCENE_LIST = [os.path.splitext(os.path.basename(s))[0] for s in SCENE_LIST]

In [ ]:
# submit jobs
PROJECT_NAME = "AN-AU-coreg-test-20m-19-12-2024"
DOWNLOAD_FOLDER = "./data/asf/"
rtc_jobs = sdk.Batch()
for g in SCENE_LIST:
    rtc_jobs += hyp3.submit_rtc_job(
        g,
        include_dem=True,  # include dem in final product
        include_inc_map=True,  # include dem map in final product
        include_rgb=True,  # include rgb img in final product
        include_scattering_area=True,  # include scat area in final product
        name=PROJECT_NAME,
        resolution=20,
        dem_name="copernicus",
        radiometry="gamma0",
    )
print(rtc_jobs)

In [ ]:
# watch jobs
rtc_jobs = hyp3.find_jobs(name=PROJECT_NAME)
rtc_jobs = hyp3.watch(rtc_jobs)

In [ ]:
# check jobs completed
print(PROJECT_NAME)
rtc_jobs = hyp3.find_jobs(name=PROJECT_NAME)
rtc_jobs = hyp3.refresh(rtc_jobs)
running_jobs = rtc_jobs.filter_jobs(succeeded=False, running=True, failed=False)
print(f"Number of running jobs: {len(running_jobs)}")
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f"Number of succeeded jobs: {len(succeeded_jobs)}")
failed_jobs = rtc_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f"Number of failed jobs: {len(failed_jobs)}")

In [ ]:
# download jobs
download_jobs = succeeded_jobs
for i, job in enumerate(download_jobs):
    print(f"scene {i+1} of {len(download_jobs)}")
    # download
    file_list = job.download_files(DOWNLOAD_FOLDER)
    process_time = job.processing_times